In [5]:
#system 'gem install bloomer --user-install'
require 'zlib'
require 'msgpack'
require 'digest/md5'
require 'bloomer/msgpackable'
require 'bloomer'
$source_vocabulary_to_fhir_url = {
    'RXNORM'=> 'http://www.nlm.nih.gov/research/umls/rxnorm',
    'LNC'=> 'http://loinc.org',
    'SNOMEDCT_US'=> 'http://snomed.info/sct',
    'NCI_UCUM'=> 'http://unitsofmeasure.org',
    'CPT'=> 'http://www.ama-assn.org/go/cpt',
    'NDFRT'=> 'http://hl7.org/fhir/ndfrt',
    'CVX'=> 'http://hl7.org/fhir/sid/cvx',
    'DSM-5'=> 'http://hl7.org/fhir/sid/dsm5',
    'ICD9CM'=> 'http://hl7.org/fhir/sid/icd-9-cm',
    'ICD10CM'=> 'http://hl7.org/fhir/sid/icd-10',
    'ICF'=> 'http://hl7.org/fhir/sid/icf-nl',
    'ATC'=> 'http://www.whocc.no/atc',
    'NUCCPT'=> 'http://nucc.org/provider-taxonomy',
    'OMIM'=> 'http://www.omim.org'
}

count = 0
$count_by_source_vocabulary = Hash.new(0)
$all_by_source_vocabulary = Hash.new { |h, k| h[k] = [] }

idx_conso_cui = 
idx_conso_cui = 
idx_termtype = 2
idx_source_vocabulary = 11
idx_source_tty = 12
idx_source_code = 13
idx_display = 14

10

10

In [12]:
count =  0
all_codes = []
for infile in Dir["scratch/2018AA-full/2018AA/META/MRCONSO.RRF.*.gz"]
  puts "test", infile
  gz = Zlib::GzipReader.new(open(infile))
  gz.each_line do |line|
    parts = line.split("|")
    # only keep preferred terms
    if parts[idx_termtype] != 'P' 
      next
    end
    puts parts
    if count > 10
      break
    end
    $count_by_source_vocabulary[parts[idx_source_vocabulary]] += 1
    if $source_vocabulary_to_fhir_url.key?(parts[idx_source_vocabulary])
      coding_system = $source_vocabulary_to_fhir_url[parts[idx_source_vocabulary]]
      coding_code = parts[idx_source_code]
      coding_display = parts[idx_display]
      tty = parts[idx_source_tty]
      $all_by_source_vocabulary[parts[idx_source_vocabulary]].push({
        :system => coding_system,
        :code => coding_code,
        :display => coding_display,
        :tty => parts[idx_source_tty]
      })
    end
    count +=1
  end
end


def include_codes_in_filter(vs_array, vocab_key, property, values)
  c=0
  $all_by_source_vocabulary.each do |one_vocab_key, terms|
    next if vocab_key != '*' and vocab_key != one_vocab_key
    terms.each do |coding|
      c += 1    
      if values == "*" or values.include? coding[property]
        vs_array.push(coding)
      end
    end
  end
end

$count_by_source_vocabulary.each do |k,c|
  if $source_vocabulary_to_fhir_url.key?(k)
    puts "#{k}\t\t\t#{c}"
  end
end

{}

In [6]:
require 'bloomer'

false

In [ ]:
all_by_source_vocabulary["OK"].push(5)

NameError: undefined local variable or method `all_by_source_vocabulary' for main:Object

In [ ]:
all_by_source_vocabulary.each {|t,v| puts(t,v)}

NameError: undefined local variable or method `all_by_source_vocabulary' for main:Object

In [ ]:
require 'zlib'

idx_rel_parent_cui = 0
idx_rel_parent_aui = 1
idx_rel_child_cui = 4
idx_rel_child_aui = 5
idx_rel_name = 7
count=0
for infile in Dir["scratch/2018AA-full/2018AA/META/MRREL.RRF.*.gz"]
  puts "load rels from", infile
  gz = Zlib::GzipReader.new(open(infile))
  gz.each_line do |line|
    parts = line.split("|")
    # only keep preferred terms
    count += 1
    if count > 10
      break
    end
    
    puts "#{parts[idx_rel_parent_cui]}/#{parts[idx_rel_parent_aui]} #{parts[idx_rel_name]} #{parts[idx_rel_child_cui]}/#{parts[idx_rel_child_aui]}"

  end
end


In [1]:
require "sqlite3"
db = SQLite3::Database.open "output/umls.sqlite"


#<SQLite3::Database:0x000056019e83db00 @tracefunc=nil, @authorizer=nil, @encoding=nil, @busy_handler=nil, @collations={}, @functions={}, @results_as_hash=nil, @type_translation=nil, @type_translator=#<Proc:0x000056019e83f798@/home/jmandel/.gem/ruby/2.5.0/gems/sqlite3-1.4.1/lib/sqlite3/database.rb:722 (lambda)>, @readonly=false>

In [ ]:
[5,false].any? do |e|
  e==5
end

In [2]:
matches = db.execute "
  select c.sab, c.tty, c.code, c.str, r.rela,
  c2.sab, c2.tty, c2.code, c2.str
  from
    MRCONSO c
    join mrrel r on c.aui=r.aui2
    join mrconso c2 on c2.aui=r.aui1
  where 
    c.code='74400008' and c.sab='SNOMEDCT_US' and c.tty='PT' and r.rel='CHD'
"
matches.each {|x| puts x}

["SNOMEDCT_US", "PT", "74400008", "Appendicitis", "isa", "SNOMEDCT_US", "PT", "24526004", "Inflammatory bowel disease"]
["SNOMEDCT_US", "PT", "74400008", "Appendicitis", "isa", "SNOMEDCT_US", "PT", "302168000", "Inflammation of large intestine"]
["SNOMEDCT_US", "PT", "74400008", "Appendicitis", "isa", "SNOMEDCT_US", "PT", "363169009", "Inflammation of specific body organs"]
["SNOMEDCT_US", "PT", "74400008", "Appendicitis", "isa", "SNOMEDCT_US", "PT", "18526009", "Disorder of appendix"]


[["SNOMEDCT_US", "PT", "74400008", "Appendicitis", "isa", "SNOMEDCT_US", "PT", "24526004", "Inflammatory bowel disease"], ["SNOMEDCT_US", "PT", "74400008", "Appendicitis", "isa", "SNOMEDCT_US", "PT", "302168000", "Inflammation of large intestine"], ["SNOMEDCT_US", "PT", "74400008", "Appendicitis", "isa", "SNOMEDCT_US", "PT", "363169009", "Inflammation of specific body organs"], ["SNOMEDCT_US", "PT", "74400008", "Appendicitis", "isa", "SNOMEDCT_US", "PT", "18526009", "Disorder of appendix"]]

In [ ]:
matches = db.execute "
with auis as (with recursive 
  ancestor_of_base(parent_aui) as (
    select c.aui
    from
      MRCONSO c
    where 
      c.code='74400008' and c.sab='SNOMEDCT_US' and c.tty='PT'
    union all 
      select r.aui1 from mrrel r join ancestor_of_base a on a.parent_aui=r.aui2 where r.rel='CHD'
  )
  select distinct parent_aui from ancestor_of_base limit 40
)
select c.aui, c.code,c.sab,c.str from auis
join MRCONSO c on c.aui=auis.parent_aui
order by str
"
matches.each {|x| puts x}

In [ ]:
root_aui = db.execute("
  select c.aui
    from
      MRCONSO c
    where 
      c.code='404684003' and c.sab='SNOMEDCT_US' and c.tty='PT'
")[0][0]

require 'set'
ancestor_pool = Set.new()
ancestor_pool

new_next_ancestors = Set.new([root_aui])

# Generate subsumption hierarchy for SNOMED


In [ ]:
class AncestryBuilder
  
  attr_accessor :sab, :system, :parents, :ancestry

  def initialize(sab, db)

    @sab = sab
    @system = $source_vocabulary_to_fhir_url[sab]

    @parents =  Hash.new {|h, k| h[k] = Set.new()}
    @ancestry = Hash.new {|h, k| h[k] = Set.new()}

    rels = db.execute("
      select c1.code, c2.code
      from
        mrrel r
        join mrconso c1 on c1.aui=r.aui1
        join mrconso c2 on c2.aui=r.aui2
      where r.rel='CHD' and r.SAB=?
      ",@sab) do |row|    
      rparent,rchild = row
      @parents[rchild].add(rparent)
    end
    
    puts "Selected #{@sab} rels from db"
    
    aa_cached = {}

    @parents.keys.each do |root_code|
      @ancestry[root_code] = if aa_cached[root_code]
        aa_cached[root_code]
      else
        count=0
        ancestor_pool = Set.new()
        new_next_ancestors = Set.new([root_code])
        until new_next_ancestors.empty? do
          count+=1
          next_ancestors = Set.new()
          new_next_ancestors.each do |code|
            next_ancestors += @parents[code]
          end
          new_next_ancestors = next_ancestors - ancestor_pool
          ancestor_pool += new_next_ancestors
        end
        aa_cached[root_code] = ancestor_pool
        ancestor_pool
      end
    end
    puts "Populated ancestry for #{@sab}"  
  end

  def to_coding(code, display=nil)
    return {
        :system => @system,
        :code => code,
        :display => display}.compact
  end

  def subsumed_by(ancestor_code)
    count = 0
    members = Set.new()
    @ancestry.each do |code, ancestor_set|
      if ancestor_set.include? ancestor_code
        count +=1
        members.add(to_coding(code))
      end    
    end
    return members
  end
end

t1 = Time.now
snomed_hierarchy = AncestryBuilder.new("SNOMEDCT_US", db)
t2 = Time.now
puts t2-t1

Selected SNOMEDCT_US rels from db


Interrupt: 

In [ ]:
x={5=>1}
if x[5]
  3
else
  2
end

In [8]:
snomed_hierarchy.subsumed_by("404684003").take(10)

[{:system=>"http://snomed.info/sct", :code=>"236016008"}, {:system=>"http://snomed.info/sct", :code=>"14053009"}, {:system=>"http://snomed.info/sct", :code=>"162068007"}, {:system=>"http://snomed.info/sct", :code=>"300402000"}, {:system=>"http://snomed.info/sct", :code=>"300403005"}, {:system=>"http://snomed.info/sct", :code=>"8644006"}, {:system=>"http://snomed.info/sct", :code=>"271835004"}, {:system=>"http://snomed.info/sct", :code=>"248490000"}, {:system=>"http://snomed.info/sct", :code=>"128050000"}, {:system=>"http://snomed.info/sct", :code=>"65576009"}]

In [84]:
class ConceptBuilder
  def initialize(sab, db)
    @sab = sab
    @system = $source_vocabulary_to_fhir_url[sab]
    @db = db
  end
  
  def to_coding(code, display=nil)
    return {
        :system => @system,
        :code => code,
        :display => display
      }.compact
  end

  def query_types(type_list)
    matching_concepts = Set.new
    type_list.each do |t|
      by_type = @db.execute("select code from MRCONSO where sab=? and tty=?", [@sab, t])
      matching_concepts += by_type.map {|cl| to_coding(cl[0])}
    end
    return matching_concepts
  end
  
  def query_all()
    matching_concepts = Set.new
    by_type = @db.execute("select code from MRCONSO where sab=?", [@sab])
    matching_concepts += by_type.map {|cl| to_coding(cl[0])}
    return matching_concepts
  end
  
  def query_types_in(type_list, code_list)
    matching_concepts = Set.new
    type_breaks = ("?" * type_list.size).split(//).join(", ")
    code_breaks = ("?" * code_list.size).split(//).join(", ")

    q = "select code, str, tty from MRCONSO where sab=? and tty in (#{type_breaks}) and code in (#{code_breaks})"
    by_type = @db.execute(q, [@sab, *type_list, *code_list])
    matching_concepts += by_type.map {|cl| to_coding(cl[0], cl[1])}
    return matching_concepts
  end
end
  
#ConceptBuilder.new("RXNORM", db).query_types(["GPCK", "BPCK"]).take(10)
#ConceptBuilder.new("RXNORM", db).query_all().take(10)
#ConceptBuilder.new("LNC", db).query_all().take(10)

:query_types_in

In [94]:
# From https://www.cms.gov/Outreach-and-Education/Medicare-Learning-Network-MLN/MLNMattersArticles/Downloads/MM10481.pdf

cms_cpt_code_description = """
Magnetic Resonance Imaging
70336, 70540, 70542, 70543, 70544, 70545, 70546, 70547, 70548, 70549, 70551, 70552,
70553, 70554, 70555, 71550, 71551, 71552, 71555, 72141, 72142, 72146, 72147, 72148,
72149, 72156, 72157, 72158, 72159, 72195, 72196, 72197, 72198, 73218, 73219, 73220,
73221, 73222, 73223, 73225, 73718, 73719, 73720, 73721, 73722, 73723, 73725, 74181,
74182, 74183, 74185, 75557, 75559, 75561, 75563, 75565, 76498
Computerized Tomography
70450, 70460, 70470, 70480, 70481, 70482, 70486, 70487, 70488, 70490, 70491, 70492,
70496, 70498, 71250, 71260, 71270, 71275, 72125, 72126, 72127, 72128, 72129, 72130,
72131, 72132, 72133, 72191, 72192, 72193, 72194, 73200, 73201, 73202, 73206, 73700,
73701, 73702, 73706, 74150, 74160, 74170, 74174, 74175, 74176, 74177, 74178, 74261,
74262, 74712, 74713, 75571, 75572, 75573, 75574, 75635, 76380, 76497,
Single-Photon Emission Computed Tomography
76390
Nuclear Medicine
78012, 78013, 78014, 78015, 78016, 78018, 78020, 78070, 78071, 78072, 78075, 78099,
78102, 78103, 78104, 78110, 78111, 78120, 78121, 78122, 78130, 78135, 78140, 78185,
78191, 78195, 78199, 78201, 78202, 78205, 78206, 78215, 78216, 78226, 78227, 78230,
78231, 78232, 78258, 78261, 78262, 78264, 78265, 78266, 78267, 78268, 78270, 78271,
78272, 78278, 78282, 78290, 78291, 78299, 78300, 78305, 78306, 78315, 78320, 78350,
78351, 78399, 78414, 78428, 78445, 78451, 78452, 78453, 78454, 78456, 78457, 78458,
78459, 78466, 78468, 78469, 78472, 78473, 78481, 78483, 78491, 78492, 78494, 78496,
78499, 78579, 78580, 78582, 78597, 78598, 78599, 78600, 78601, 78605, 78606, 78607,
78608, 78609, 78610, 78630, 78635, 78645, 78647, 78650, 78660, 78699, 78700, 78701,
78707, 78708, 78709, 78710, 78725, 78730, 78740, 78761, 78799, 78800, 78801, 78802,
78803, 78804, 78805, 78806, 78807, 78811, 78812, 78813, 78814, 78816, 78999
"""

cms_cpt_codes = cms_cpt_code_description.scan(/\d+/)


cms_cpt_codes.length

247

In [100]:
cpt_for_pama = ConceptBuilder.new("CPT", db).query_types_in(["PT"], cms_cpt_codes)
write_filter_to_file(cpt_for_pama, "./output/cpt-for-pama")

640

In [95]:
breaks = ("?" * cms_cpt_codes.size).split(//).join(", ")
q = "select code, str, tty from MRCONSO where sab=? and tty='PT' and code in (#{breaks})"
db.execute(q, ["CPT", *cms_cpt_codes]).length



247

In [99]:
require 'fhir_dstu2_models'

$ancestry_builders = {
  'SNOMEDCT_US' => snomed_hierarchy
  }

def codes_for_valueset(vs_argonaut_codes, db)
  

  vs_set = Set.new
  puts "Num includes: #{vs_argonaut_codes.compose.include.count}"
  apply_filters = lambda do |vs_element, logtag|
    
    puts "include #{vs_element}: #{logtag}"
    required_system = vs_element.system
    raise "don't know system" unless $source_vocabulary_to_fhir_url.values.include?(required_system)

    required_key = ($source_vocabulary_to_fhir_url.select {|k,v| v == required_system}).keys[0]
    puts required_key
    raise "can't handle filter intersection" unless vs_element.filter.count <= 1
    
    if vs_element.filter.count == 0
      return ConceptBuilder.new(required_key, db).query_all()
    else
      filter = vs_element.filter[0]
      if filter.op == "in"
        return ConceptBuilder.new(required_key, db).query_types(filter.value.split(","))
      elsif filter.op == "is-a"
        return $ancestry_builders[required_key].subsumed_by(filter.value)
      end
    end
    puts "filter done with size: #{vs_set.count}"    
  end
  
  vs_argonaut_codes.compose.include.each do |vs_include|
    vs_set += apply_filters.(vs_include, :include)
  end
  
  vs_argonaut_codes.compose.exclude.each do |vs_exclude|
    vs_set -= apply_filters.(vs_exclude, :exclude)
  end
  
  puts vs_set.count
  return vs_set
end


def write_filter_to_file(vs_set, output_file)
  vs_array = vs_set.to_a.map do |v|
      {
        :system => v[:system],
        :code => v[:code],
        :display => v[:display]

        }
  end
    
  Zlib::GzipWriter.open(output_file+".json.gz") { |gz| gz.write ({
    "resourceType"=> "ValueSet",
    "expansion"=> {
      "contains" =>vs_array
      } 
  }).to_json}

  expected_size = 100
  false_positive_probability = 0.001

  vs_bloom = Bloomer::Scalable.new(expected_size, false_positive_probability)
  vs_array.each do |coding|
    vs_bloom.add "#{coding[:system]}|#{coding[:code]}"
  end
  
  File.open(output_file+".msgpck", "w") do |f|
    f.write(vs_bloom.to_msgpack())
  end
end

:write_filter_to_file

In [ ]:

all_umls_terms = Set.new
$source_vocabulary_to_fhir_url.keys.each do |k|
  puts k
  these_terms = ConceptBuilder.new(k, db).query_all()
  write_filter_to_file(these_terms, "./output/all-#{k.downcase}")
  all_umls_terms += these_terms
  manifest["filters"] << {
    "codeSystemUrl" => $source_vocabulary_to_fhir_url[k],
    "probability"=> 0.001,
    "file"=> "all-#{k.downcase}.msgpck"
  }

end
write_filter_to_file(all_umls_terms, "./output/all-umls-terms")

#       coding_system = $source_vocabulary_to_fhir_url[parts[idx_source_vocabulary]]
#       coding_code = parts[idx_source_code]
#       coding_display = parts[idx_display]
#       tty = parts[idx_source_tty]
#       $all_by_source_vocabulary[parts[idx_source_vocabulary]].push({
#         :system => coding_system,
#         :code => coding_code,
#         :display => coding_display,
#         :tty => parts[idx_source_tty]
#       })


In [ ]:
manifest = {
  "filters"=> []
  }

[
  "daf-problem.canonical",
  "substance-sct",
  "medication-codes",
  "observation-codes",
  "observation-value-codes",
  "procedure-type"].each do |vs_json_file|
    vs = open("./inferno/resources/argonauts/ValueSet-#{vs_json_file}.json").read()
    vs = FHIR::DSTU2::Json.from_json(vs)
    vs_set = codes_for_valueset(vs, db)
    vs_url = vs.url
    write_filter_to_file(vs_set, "./output/valueset-#{vs_json_file}")
    manifest["filters"] << {
      "valueSetUrl" => vs_url,
      "probability"=> 0.001,
      "file"=> "valueset-#{vs_json_file}.msgpck"
    }
end

open("./output/manifest.json", "w") { |mfile| mfile.write(JSON.pretty_generate manifest) }


0

In [ ]:

open("./output/manifest.json", "w") { |mfile| mfile.write(JSON.pretty_generate manifest) }

In [ ]:
Set.new([{:a=> 1}]) + (Set.new([{:b=> 1}]))

In [ ]:
def a
  5
end

a#to_proc()

In [ ]:
db.execute("
  select
  -- c.cui, c.aui, c.code, c.sab, c.str
  *
  from mrconso c
  join  MRREL r on r.aui1=c.aui
  where r.aui2=?
  and rel='PAR'
  limit 1000",
[aui])

In [ ]:
require 'fhir_dstu2_models'
require 'bloomer'


def write_filter_to_file(vs_array, output_file)
  Zlib::GzipWriter.open(output_file+".json.gz") { |gz| gz.write ({
    "resourceType"=> "ValueSet",
    "expansion"=> vs_array.map do |v|
      {
        :system => v[:system],
        :code => v[:code],
        :display => v[:display]
        }
    end
    }).to_json }

  expected_size = 100
  false_positive_probability = 0.0001

  vs_bloom = Bloomer::Scalable.new(expected_size, false_positive_probability)
  vs_array.each do |coding|
    vs_bloom.add "#{coding[:system]}|#{coding[:code]}"
  end
  
  File.open(output_file+".msgpck", "w") do |f|
    f.write(vs_bloom.to_msgpack())
  end
end

def read_relations()
end

def bloom_filter_for_valueset(valueset_filename, output_file)
  vs_argonaut_codes_str = open(valueset_filename).read()
  vs_argonaut_codes = FHIR::DSTU2::Json.from_json(vs_argonaut_codes_str)
  vs_array = []

  vs_argonaut_codes.compose.include.each do |vs_include|
    required_system = vs_include.system
    raise "don't know system" unless $source_vocabulary_to_fhir_url.values.include?(required_system)

    required_key = ($source_vocabulary_to_fhir_url.select {|k,v| v == required_system}).keys[0]
    puts required_key
    raise "can't handle filter intersection" unless vs_include.filter.count <= 1
    if vs_include.filter.count == 0
      filter_property = nil
      filter_values = "*"
    else
      filter = vs_include.filter[0]
      if filter.op == "in"
        filter_property = filter.property.downcase.to_sym
        filter_values = filter.value.split(",")
      else
        puts "backing out for #{output_file} because of op"
        filter_values = "*"
      end
    end 
    puts(filter_property, filter_values)
    include_codes_in_filter(vs_array, required_key, filter_property, filter_values)
  end
  write_filter_to_file(vs_array, output_file)
  puts "#{output_file}: #{vs_array.count}"
end

bloom_filter_for_valueset("./inferno/resources/argonauts/ValueSet-medication-codes.json", "ValueSet-medication-codes")
bloom_filter_for_valueset("./inferno/resources/argonauts/ValueSet-observation-codes.json", "ValueSet-observation-codes")
bloom_filter_for_valueset("./inferno/resources/argonauts/ValueSet-procedure-type.json", "ValueSet-procedure-type")

RXNORM
tty
["SCD", "SBD", "GPCK", "BPCK", "SCDG", "SBDG", "SCDF", "SBDF"]


NoMethodError: undefined method `include_codes_in_filter' for main:Object

In [ ]:
vs_array = []
include_codes_in_filter(vs_array, "SNOMEDCT_US", :tty, "*")
puts vs_array.count
write_filter_to_file(vs_array, "sct-us")


vs_array = []
include_codes_in_filter(vs_array, "RXNORM", :tty, "*")
puts vs_array.count
write_filter_to_file(vs_array, "rxnorm-all")


vs_array = []
include_codes_in_filter(vs_array, "*", :tty, "*")
puts vs_array.count
write_filter_to_file(vs_array, "umls-all")

In [ ]:
rehydrated = Bloomer::Scalable.from_msgpack(open("ValueSet-procedure-type.msgpck").read())
puts rehydrated.count, 26**4

chars = 3
('a'*chars..'z'*chars).map do |n|
  if rehydrated.include?("#{n}")
    puts n
    1
  else
    0
  end
end.sum * 1.0 / (26**chars)

In [70]:
{
  :a => 1,
  :b => nil
  }.compact

{:a=>1}